![header](../img/header.jpg)

# Exercise 1: Flopy, MODFLOW, and MODPATH
The purpose of this exercise is to use Flopy to simulate example problem 1 using MODFLOW-2005, MODPATH, and MT3D.

## Part I. Setup Notebook

In [ ]:
%matplotlib inline
from __future__ import print_function
import sys
import os
import shutil
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import flopy

import config

print(sys.version)
print('python executable: {}'.format(sys.executable))
print('numpy version: {}'.format(np.__version__))
print('matplotlib version: {}'.format(mpl.__version__))
print('flopy version: {}'.format(flopy.__version__))

## Part II. Create, Run, and Post-Process MODFLOW-2005 Model

In [ ]:
# model info
model_name = 'ex01'
model_ws = './ex01-completed'

# grid properties
nlay = 3
nrow = 21
ncol = 20
delr = 500.
delc = 500.
top = 400.
botm = [220., 200., 0.]

# hydraulic properties
hk0 = 50.
vk0 = 10.
hk1 = 0.01
vk1 = 0.01
hk2 = 200.
vk2 = 20

# boundary conditions
welspd = [3 - 1, 11 - 1, 10 - 1, -1.5e5]
rech = 0.005
rivspd = [[0, ir, ncol-1, 320., 1.e5, 318.] for ir in range(nrow)]

In [ ]:
m = flopy.modflow.Modflow(modelname=model_name, model_ws=model_ws, exe_name=config.mfexe)
dis = flopy.modflow.ModflowDis(m, nlay, nrow, ncol, delr=delr, delc=delc, top=top, botm=botm)
bas = flopy.modflow.ModflowBas(m)
lpf = flopy.modflow.ModflowLpf(m, hk=[hk0, hk1, hk2], vka=[vk0, vk1, vk2])
wel = flopy.modflow.ModflowWel(m, stress_period_data=welspd)
rch = flopy.modflow.ModflowRch(m, rech=rech)
riv = flopy.modflow.ModflowRiv(m, stress_period_data=rivspd)
pcg = flopy.modflow.ModflowPcg(m)
lmt = flopy.modflow.ModflowLmt(m)
oc  = flopy.modflow.ModflowOc(m, stress_period_data={(0, 0): ['SAVE HEAD', 'SAVE BUDGET']})
oc.reset_budgetunit()
m.write_input()
m.run_model()

In [ ]:
# load the heads and budget files
fname = os.path.join(model_ws, model_name + '.hds')
hdobj = flopy.utils.HeadFile(fname)
head = hdobj.get_data()

fname = os.path.join(model_ws, model_name + '.cbc')
cbb = flopy.utils.CellBudgetFile(fname)
frf = cbb.get_data(text='FLOW RIGHT FACE')[0]
fff = cbb.get_data(text='FLOW FRONT FACE')[0]

In [ ]:
# plot the results for each layer
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 8))
vmin = head.min()
vmax = head.max()
levels = np.linspace(vmin, vmax, 20)

for ilay in range(nlay):
    ax = axes[ilay]
    ax.set_title('Layer {}'.format(ilay + 1))
    ax.set_aspect('equal')
    modelmap = flopy.plot.PlotMapView(model=m, layer=ilay, ax=ax)
    quadmesh = modelmap.plot_array(head, masked_values=[999.], alpha=0.5, 
                                   vmin=vmin, vmax=vmax, cmap='jet')
    linecollection = modelmap.plot_grid()
    contour_set = modelmap.contour_array(head, levels=levels, colors='white')
    quiver = modelmap.plot_discharge(frf, fff, head=head)
cb = fig.colorbar(quadmesh, ax=axes, shrink=0.5)

## Part II. Create and Run MODPATH-7 Model

In [ ]:
# particles
partlocs = [(0, i, 0) for i in range(nrow)]
particledata = flopy.modpath.ParticleData(partlocs, structured=True)
pg = flopy.modpath.ParticleGroup(particledata=particledata)

In [ ]:
mp = flopy.modpath.Modpath7(modelname='ex01_mp', model_ws=model_ws, flowmodel=m, exe_name=config.mpexe)
mpbas = flopy.modpath.Modpath7Bas(mp)
mpsim = flopy.modpath.Modpath7Sim(mp, particlegroups=pg)
mp.write_input()
mp.run_model()

In [ ]:
# load the pathline results
fname = os.path.join(model_ws, 'ex01_mp.mppth')
p = flopy.utils.PathlineFile(fname).get_alldata()

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(1, 1, 1)
mm = flopy.plot.PlotMapView(model=m)
mm.plot_grid(lw=0.5)
mm.plot_pathline(p, layer='all', colors='k', label='Particle Paths')
mm.ax.legend();

## Part III. Create and Run MT3D Model

In [ ]:
# transport boundary conditions
icbund = np.ones((nlay, nrow, ncol), dtype=np.int)
icbund[0, int(nrow / 2), 0] = -1
sconc = np.zeros((nlay, nrow, ncol), dtype=np.float32)
sconc[0, int(nrow / 2), 0] = 100

# observations
rhalf = int(nrow / 2)
chalf = int(ncol / 2)
obs = [(il, rhalf, chalf) for il in range(nlay)]

In [ ]:
modelname = 'ex01_mt3d'
mt = flopy.mt3d.Mt3dms(modelname=modelname, model_ws=model_ws, modflowmodel=m, exe_name=config.mtexe)
btn = flopy.mt3d.Mt3dBtn(mt, icbund=icbund, sconc=sconc, perlen=100000., nprs=-1, obs=obs)
adv = flopy.mt3d.Mt3dAdv(mt, mixelm=0)
dsp = flopy.mt3d.Mt3dDsp(mt, al=10.)
ssm = flopy.mt3d.Mt3dSsm(mt)
gcg = flopy.mt3d.Mt3dGcg(mt)
mt.write_input()
mt.run_model(silent=False, normal_msg='Program completed.')

In [ ]:
# load the concentrations
fname = os.path.join(model_ws, 'MT3D001.UCN')
cobj = flopy.utils.UcnFile(fname)
conc = cobj.get_data()

# load observations
fname = os.path.join(model_ws, 'MT3D001.OBS')
cobs = flopy.mt3d.Mt3dms.load_obs(fname)

In [ ]:
# plot the results for each layer
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 8))
a = conc
a = np.ma.masked_less(a, 1.e-4)
vmin = a.min()
vmax = a.max()
levels = [0.01, .1, 1, 10., 100.]

for ilay in range(nlay):
    ax = axes[ilay]
    ax.set_title('Layer {}'.format(ilay + 1))
    ax.set_aspect('equal')
    modelmap = flopy.plot.PlotMapView(model=m, layer=ilay, ax=ax)
    quadmesh = modelmap.plot_array(a, masked_values=[999.], alpha=0.5, 
                                   vmin=vmin, vmax=vmax, cmap='jet')
    linecollection = modelmap.plot_grid()
    contour_set = modelmap.contour_array(a, levels=levels, colors='k')
    quiver = modelmap.plot_discharge(frf, fff, head=head)
cb = fig.colorbar(quadmesh, ax=axes, shrink=0.5)

In [ ]:
fig = plt.figure(figsize=(8, 3))
ax = fig.add_subplot(1, 1, 1)
modelxsect = flopy.plot.PlotCrossSection(model=m, line={'row': rhalf})
linecollection = modelxsect.plot_grid()
modelxsect.plot_array(a, head=head, cmap='jet')
t = ax.set_title('Column {}'.format(rhalf + 1))

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
for name in cobs.dtype.names:
    if name.startswith('('):
        ax.plot(cobs['time'], cobs[name], label=name)
ax.legend()
ax.set_xlabel('Time')
ax.set_ylabel('Concentration')
ax.set_title('Simulated Breakthrough Curve');